In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

In [2]:
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.23.1
tensorflow 2.3.1
tensorflow.keras 2.4.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [4]:
"""
归一化 x = (x - u) / std  u:均值，std:方差
"""
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#x_train:[None, 28, 28] -> [None, 784]
x_train_scaler = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28) 
x_valid_scaler = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)   
x_test_scaler = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)   

In [6]:
# tf.keras.models.Sequential()

"""
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
"""

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# relu: y = max(0, x)
# softmax: 将向量变成概率分布. x = [x1, x2, x3], 
#          y = [e^x1/sum, e^x2/sum, e^x3/sum], sum = e^x1 + e^x2 + e^x3

# reason for sparse: y->index. y->one_hot->[] 
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(0.001),
              metrics = ["accuracy"])

In [8]:
# Tensorboard, earlystopping, Modelcheckpoint
logdir = "./callbacks"
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, "fashion_mnist_model.h5")
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]

history = model.fit(x_train_scaler, y_train, epochs=50,
                    validation_data=(x_valid_scaler, y_valid),
                    callbacks=callbacks)

Epoch 1/50
1719/1719 [==============================] - 1s 792us/step - loss: 0.3860 - accuracy: 0.8633 - val_loss: 0.3867 - val_accuracy: 0.8642
Epoch 2/50
1719/1719 [==============================] - 1s 713us/step - loss: 0.3793 - accuracy: 0.8652 - val_loss: 0.3799 - val_accuracy: 0.8698
Epoch 3/50
1719/1719 [==============================] - 1s 708us/step - loss: 0.3729 - accuracy: 0.8676 - val_loss: 0.3806 - val_accuracy: 0.8628
Epoch 4/50
1719/1719 [==============================] - 1s 713us/step - loss: 0.3674 - accuracy: 0.8690 - val_loss: 0.3711 - val_accuracy: 0.8710
Epoch 5/50
1719/1719 [==============================] - 1s 712us/step - loss: 0.3623 - accuracy: 0.8710 - val_loss: 0.3678 - val_accuracy: 0.8702
Epoch 6/50
1719/1719 [==============================] - 1s 710us/step - loss: 0.3571 - accuracy: 0.8730 - val_loss: 0.3642 - val_accuracy: 0.8698
Epoch 7/50
1719/1719 [==============================] - 1s 714us/step - loss: 0.3524 - accuracy: 0.8749 - val_loss: 0.3609 -

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaler, y_test)